# Import and initialization

In [1]:
import json
import os
import random
import requests
import numpy as np

from datetime import datetime

In [2]:
from checklist.test_suite import TestSuite
from checklist.test_types import MFT, INV, DIR

In [3]:
INTENTS = [
    'book-appointment',
    'book-hotel',
    'book-restaurant',
    'change-appointment',
    'change-car-rental',
    'change-driver',
    'change-hotel',
    'change-reminder',
    'change-restaurant',
    'contact-provider',
    'context',
    'find-Property',
    'find-Recipe',
    'find-Trip',
    'find-activity',
    'find-around-me',
    'find-car-rental',
    'find-driver',
    'find-flight',
    'find-hotel',
    'find-information',
    'find-itinerary',
    'find-restaurant',
    'find-train',
    'inform',
    'irrelevant',
    'other',
    'provide-agenda',
    'provide-news',
    'provide-showtimes',
    'provide-translation',
    'provide-tv-guide',
    'provide-weather',
    'purchase',
    'set-reminder',
    'smalltalk',
    'unsupported',
]

INTENTS_REDUCT = [
    'find-around-me',
    'find-flight',
    'find-hotel',
    'find-restaurant',
    'find-train',
    'irrelevant',
    'provide-showtimes',
    'purchase',
]

# Tests creation

In [4]:
FORCE = 1

In [16]:
with open('data/dataset_final.json', "r") as f: #TODO: adapt to the new name, for example for forces
    data = json.load(f)
random.shuffle(data)

In [6]:
types_aug = list(data[0].keys())
types_aug.remove('intent')
types_aug.remove('sentence')
print(f'Types of augmentation : {types_aug}')

Types of augmentation : ['keyboard_augmentation', 'letter_augmentation', 'letter_deletion', 'letter_swap', 'bert_swap']


In [7]:
def create_tests_aug(type_aug):
    data_aug = { "data": [elem.get(type_aug) for elem in data],
                 "labels": [INTENTS.index(elem.get("intent")) for elem in data]}
    assert(data_aug.get('data')[0] != None)
    
    # MFT tests
    test_MFT_aug = MFT(**data_aug, name=f"MFT {type_aug}", capability=type_aug, description="")

    # INV test
    sentences_original = data_original.get("data")
    sentences_aug = data_aug.get("data")
    t = { "data": [[sentences_original[i], sentences_aug[i]] for i in range(len(sentences_original))]}
    test_INV_aug = INV(**t, name=f"INV {type_aug}", capability=type_aug, description="")

    suite.add(test_MFT_aug, overwrite=True)
    suite.add(test_INV_aug, overwrite=True)

In [8]:
suite = TestSuite()

In [9]:
data_original = { "data": [elem.get('sentence') for elem in data],
                "labels": [INTENTS.index(elem.get("intent")) for elem in data]}
test_MFT_original = MFT(**data_original, name='MFT original', capability='original', description='')
suite.add(test_MFT_original)

In [10]:
for type_aug in types_aug :
    create_tests_aug(type_aug=type_aug)

# Tests running

In [11]:
def get_classif(classification):
    value = INTENTS.index(max(classification, key=classification.get))
    if value not in [15, 18, 19, 22, 23, 25, 29, 33] :
        value = 25
    return value

def get_softmax(classification):
    return [c/100 for c in classification.values()]


def get_classification(sentences):
    URL = "http://teaching-pfe.francecentral.azurecontainer.io/api/classification"  
    LANG = "fr-FR"
    classifs = [requests.get(f"{URL}?iso_language={LANG}&sentence={sentence}").json().get("classification") for sentence in sentences]
    return (np.array([get_classif(c) for c in classifs]), np.array([get_softmax(c) for c in classifs]))

In [12]:
suite.tests

OrderedDict([('MFT original', <checklist.test_types.MFT at 0x7f3e37ba73c8>),
             ('MFT keyboard_augmentation',
              <checklist.test_types.MFT at 0x7f3e37c1e828>),
             ('INV keyboard_augmentation',
              <checklist.test_types.INV at 0x7f3e37c1eb38>),
             ('MFT letter_augmentation',
              <checklist.test_types.MFT at 0x7f3e37c1ef60>),
             ('INV letter_augmentation',
              <checklist.test_types.INV at 0x7f3e589e2390>),
             ('MFT letter_deletion',
              <checklist.test_types.MFT at 0x7f3e37ba7080>),
             ('INV letter_deletion',
              <checklist.test_types.INV at 0x7f3e37ba7198>),
             ('MFT letter_swap', <checklist.test_types.MFT at 0x7f3e37ba7278>),
             ('INV letter_swap', <checklist.test_types.INV at 0x7f3e37ba7390>),
             ('MFT bert_swap', <checklist.test_types.MFT at 0x7f3e37ba75c0>),
             ('INV bert_swap', <checklist.test_types.INV at 0x7f3e37ba7518>)]

In [13]:
start = datetime.now()
print(f"Started at {start} \n")

suite.run(get_classification, overwrite=True)
    
end = datetime.now()
print(f"\nExecution time : {end - start}")

Started at 2021-01-15 16:44:47.802648 

Running MFT original
Predicting 2 examples
Running MFT keyboard_augmentation
Predicting 2 examples
Running INV keyboard_augmentation
Predicting 4 examples
Running MFT letter_augmentation
Predicting 2 examples
Running INV letter_augmentation
Predicting 4 examples
Running MFT letter_deletion
Predicting 2 examples
Running INV letter_deletion
Predicting 4 examples
Running MFT letter_swap
Predicting 2 examples
Running INV letter_swap
Predicting 4 examples
Running MFT bert_swap
Predicting 2 examples
Running INV bert_swap
Predicting 4 examples

Execution time : 0:00:25.846769


In [14]:
suite.summary()

letter_augmentation

MFT letter_augmentation
Test cases:      2
Fails (rate):    0 (0.0%)


INV letter_augmentation
Test cases:      2
Fails (rate):    0 (0.0%)




letter_swap

MFT letter_swap
Test cases:      2
Fails (rate):    0 (0.0%)


INV letter_swap
Test cases:      2
Fails (rate):    0 (0.0%)




original

MFT original
Test cases:      2
Fails (rate):    0 (0.0%)




letter_deletion

MFT letter_deletion
Test cases:      2
Fails (rate):    0 (0.0%)


INV letter_deletion
Test cases:      2
Fails (rate):    0 (0.0%)




bert_swap

MFT bert_swap
Test cases:      2
Fails (rate):    0 (0.0%)


INV bert_swap
Test cases:      2
Fails (rate):    0 (0.0%)




keyboard_augmentation

MFT keyboard_augmentation
Test cases:      2
Fails (rate):    0 (0.0%)


INV keyboard_augmentation
Test cases:      2
Fails (rate):    0 (0.0%)






# Saving tests suite

In [15]:
if not os.path.isdir("testsuite") :
    os.mkdir("testsuite")
suite.save(os.path.join("testsuite", f"suite_force{FORCE}.pkl"))